# Interactive Bayesian Governance Explorer

This notebook allows you to explore different scenarios and parameters for the Bayesian vs Checklist audit comparison.

**Use this to**:
- Test different stress scenarios
- Experiment with prior beliefs
- Understand prior-likelihood-posterior dynamics
- Visualize how quickly Bayesian updates converge

In [ ]:
# Import the simulation module
import sys
sys.path.append('.')
from bayesian_governance_simulation import *

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## Part 1: Basic Bayesian Updating

Let's start simple: observe how a single day's evidence updates your belief.

In [ ]:
def interactive_single_update():
    """Interactive widget for understanding single Bayesian update"""
    
    @widgets.interact(
        prior_failures=widgets.IntSlider(min=1, max=20, value=2, description='Prior failures (α):'),
        prior_successes=widgets.IntSlider(min=50, max=200, value=98, description='Prior successes (β):'),
        observed_failures=widgets.IntSlider(min=0, max=50, value=5, description='Observed failures:'),
        observed_total=widgets.IntSlider(min=10, max=200, value=100, description='Observed batches:')
    )
    def update_plot(prior_failures, prior_successes, observed_failures, observed_total):
        
        # Calculate distributions
        x = np.linspace(0, 0.20, 1000)
        
        # Prior
        prior_dist = stats.beta.pdf(x, prior_failures, prior_successes)
        prior_mean = prior_failures / (prior_failures + prior_successes)
        
        # Likelihood (scaled for visualization)
        likelihood = stats.binom.pmf(
            observed_failures,
            observed_total,
            x
        )
        likelihood_scaled = likelihood / likelihood.max() * prior_dist.max()
        
        # Posterior
        post_alpha = prior_failures + observed_failures
        post_beta = prior_successes + (observed_total - observed_failures)
        posterior_dist = stats.beta.pdf(x, post_alpha, post_beta)
        posterior_mean = post_alpha / (post_alpha + post_beta)
        
        # Observed rate
        observed_rate = observed_failures / observed_total
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        ax.plot(x, prior_dist, 'b--', linewidth=2, label=f'Prior: β({prior_failures}, {prior_successes})')
        ax.fill_between(x, prior_dist, alpha=0.2, color='blue')
        
        ax.plot(x, likelihood_scaled, 'g:', linewidth=2, label='Likelihood (scaled)')
        
        ax.plot(x, posterior_dist, 'r-', linewidth=3, label=f'Posterior: β({post_alpha}, {post_beta})')
        ax.fill_between(x, posterior_dist, alpha=0.3, color='red')
        
        ax.axvline(observed_rate, color='black', linestyle=':', linewidth=2, 
                  label=f'Observed: {observed_rate:.1%}')
        
        ax.axvline(prior_mean, color='blue', alpha=0.5, linestyle='--')
        ax.axvline(posterior_mean, color='red', alpha=0.5, linestyle='--')
        
        ax.set_xlabel('Failure Rate', fontsize=12)
        ax.set_ylabel('Probability Density', fontsize=12)
        ax.set_title('Bayesian Update: Prior × Likelihood → Posterior', fontsize=14, fontweight='bold')
        ax.legend(loc='upper right', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Add text box with summary
        summary = f"""Prior mean: {prior_mean:.3f}
Observed: {observed_rate:.3f}
Posterior mean: {posterior_mean:.3f}

Shift: {abs(posterior_mean - prior_mean):.3f}
        """
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.02, 0.97, summary, transform=ax.transAxes, fontsize=10,
               verticalalignment='top', bbox=props)
        
        plt.tight_layout()
        plt.show()

interactive_single_update()

### Key Observations

**Experiment with the sliders**:

1. **Strong prior + weak evidence**: Set α=10, β=90 (10% prior belief), then observe just 2 failures in 20 batches (10% observed). Notice the posterior barely moves—prior dominates.

2. **Weak prior + strong evidence**: Set α=1, β=1 (50% uninformative prior), then observe 5 failures in 100 batches (5% observed). Posterior immediately shifts to match evidence—data dominates.

3. **Conflicting evidence**: Set α=2, β=98 (2% prior), then observe 10 failures in 50 batches (20% observed). Watch the posterior compromise between prior and likelihood.

**The Bayesian Learning Principle**: 
- Prior represents **accumulated knowledge**
- Likelihood represents **new evidence**  
- Posterior is the **weighted compromise** (weight depends on relative information content)

## Part 2: Sequential Learning

Now let's see how beliefs evolve **over time** as evidence accumulates.

In [ ]:
def sequential_learning_demo(true_rate=0.05, n_days=100, batches_per_day=50, 
                            prior_alpha=2, prior_beta=98):
    """Show how Bayesian belief converges to true value over time"""
    
    np.random.seed(42)
    
    auditor = BayesianAuditor(prior_alpha=prior_alpha, prior_beta=prior_beta)
    
    pofs = [auditor.get_pof()]
    lowers = []
    uppers = []
    
    for day in range(n_days):
        # Generate observations from true process
        failures = np.random.binomial(batches_per_day, true_rate)
        
        # Update belief
        auditor.update(failures, batches_per_day)
        pofs.append(auditor.get_pof())
        
        lower, upper = auditor.get_credible_interval()
        lowers.append(lower)
        uppers.append(upper)
    
    # Plot convergence
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    days = np.arange(n_days + 1)
    
    # Top: PoF convergence
    ax1.plot(days, pofs, 'b-', linewidth=2, label='Bayesian PoF')
    ax1.axhline(true_rate, color='red', linestyle='--', linewidth=2, label='True Rate')
    ax1.axhline(pofs[0], color='blue', linestyle=':', alpha=0.5, label='Prior Mean')
    
    ax1.set_xlabel('Day', fontsize=11)
    ax1.set_ylabel('Probability of Failure', fontsize=11)
    ax1.set_title(f'Bayesian Convergence (True Rate = {true_rate:.1%})', 
                 fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Bottom: Uncertainty reduction
    uncertainty = np.array(uppers) - np.array(lowers)
    ax2.plot(days[1:], uncertainty, 'g-', linewidth=2)
    ax2.fill_between(days[1:], 0, uncertainty, alpha=0.3, color='green')
    
    ax2.set_xlabel('Day', fontsize=11)
    ax2.set_ylabel('Credible Interval Width', fontsize=11)
    ax2.set_title('Uncertainty Reduction Over Time', fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    final_error = abs(pofs[-1] - true_rate)
    initial_error = abs(pofs[0] - true_rate)
    
    print(f"\nConvergence Summary:")
    print(f"{'='*50}")
    print(f"True rate:          {true_rate:.4f}")
    print(f"Prior mean:         {pofs[0]:.4f} (error: {initial_error:.4f})")
    print(f"Final posterior:    {pofs[-1]:.4f} (error: {final_error:.4f})")
    print(f"Error reduction:    {(1 - final_error/initial_error)*100:.1f}%")
    print(f"Final uncertainty:  {uncertainty[-1]:.4f}")
    print(f"{'='*50}")

# Run demo
sequential_learning_demo(true_rate=0.05, n_days=100, batches_per_day=50)

### What to Notice

1. **Convergence speed**: PoF reaches true rate within ~20-30 days
2. **Uncertainty reduction**: Credible interval narrows as evidence accumulates
3. **Asymptotic behavior**: After sufficient data, additional days provide diminishing returns

**Try changing parameters**:
```python
sequential_learning_demo(
    true_rate=0.08,        # Different true process
    prior_alpha=10,        # Stronger prior
    prior_beta=90          # (implies 10% prior belief)
)
```

## Part 3: Scenario Explorer

Now let's run the full simulation with custom parameters.

In [ ]:
def explore_scenarios():
    """Interactive scenario explorer"""
    
    @widgets.interact(
        stress_duration=widgets.IntSlider(min=20, max=150, value=100, step=10,
                                         description='Stress duration (days):'),
        stress_intensity=widgets.FloatSlider(min=0.001, max=0.005, value=0.002, step=0.0005,
                                            description='Stress intensity:',
                                            readout_format='.4f'),
        prior_alpha=widgets.IntSlider(min=1, max=10, value=2,
                                     description='Prior α:'),
        prior_beta=widgets.IntSlider(min=50, max=150, value=98,
                                    description='Prior β:')
    )
    def run_custom_scenario(stress_duration, stress_intensity, prior_alpha, prior_beta):
        
        print(f"Running scenario with custom parameters...")
        print(f"Prior belief: {prior_alpha/(prior_alpha+prior_beta):.2%}")
        print(f"Stress period: {stress_duration} days\n")
        
        # Modify ProcessState to use custom stress intensity
        # (This is a simplified version; full implementation would subclass ProcessState)
        
        results = run_simulation(
            days=365,
            daily_batches=50,
            stress_start=150,
            stress_end=150 + stress_duration
        )
        
        plot_simulation_results(results)
        generate_summary_metrics(results)

explore_scenarios()

## Part 4: Real-World Calibration Exercise

**Challenge**: Last year's audit found 2% failure rate in EBR data integrity. 

This week you observe:
- Monday: 3 failures / 47 batches
- Tuesday: 2 failures / 51 batches  
- Wednesday: 5 failures / 48 batches
- Thursday: 4 failures / 50 batches
- Friday: 6 failures / 49 batches



In [ ]:
# Initialize with last year's audit as prior
auditor = BayesianAuditor(prior_alpha=2, prior_beta=98)

print("Starting belief (from last year's audit):")
print(f"  PoF: {auditor.get_pof():.3f}")
lower, upper = auditor.get_credible_interval()
print(f"  95% Credible Interval: [{lower:.3f}, {upper:.3f}]\n")

# Weekly observations
week_data = [
    ("Monday", 3, 47),
    ("Tuesday", 2, 51),
    ("Wednesday", 5, 48),
    ("Thursday", 4, 50),
    ("Friday", 6, 49)
]

print("Daily Updates:")
print("="*60)

for day, failures, total in week_data:
    pof = auditor.update(failures, total)
    lower, upper = auditor.get_credible_interval()
    observed_rate = failures / total
    
    print(f"{day:12} | Observed: {observed_rate:.1%} ({failures}/{total})")
    print(f"             | Updated PoF: {pof:.3f} [{lower:.3f}, {upper:.3f}]")
    print("-"*60)

# Final assessment
final_pof = auditor.get_pof()
week_avg = sum(f for _, f, _ in week_data) / sum(t for _, _, t in week_data)

print("\n" + "="*60)
print("WEEK SUMMARY:")
print("="*60)
print(f"Week average observed rate: {week_avg:.1%}")
print(f"Final Bayesian PoF:         {final_pof:.1%}")
print(f"Shift from prior:           +{(final_pof - 0.02)*100:.1f}pp")

if final_pof > 0.06:
    print("\n⚠️  CRITICAL: PoF > 6% → Immediate escalation required")
elif final_pof > 0.03:
    print("\n⚠️  WARNING: PoF > 3% → Investigate within 48 hours")
else:
    print("\n✓ Normal: PoF within baseline range")

print("="*60)

### Decision Framework

**Traditional Checklist Approach**:  
"We'll review this at next quarterly audit (60 days away)"

**Bayesian Governance Approach**:  
"PoF crossed 3% threshold on Wednesday → triggered investigation protocol → root cause identified by Friday"

**The Difference**: ~55 days of lead time to prevent manifold fracture

## Part 5: Building Intuition for Priors

**Key Question**: How do you choose α and β?

### Method 1: Pseudo-counts

Think of α and β as "imaginary" prior observations:
- α = number of failures you've "seen"
- β = number of successes you've "seen"

Example: Last year's audit sampled 100 batches, found 2 failures
→ Use `Beta(2, 98)` as prior

### Method 2: Belief + Confidence

- **Belief**: What failure rate do you expect? (sets α/(α+β))
- **Confidence**: How sure are you? (sets α+β, the "effective sample size")

Examples:
- `Beta(2, 98)`: 2% belief, low confidence (n_eff = 100)
- `Beta(20, 980)`: 2% belief, high confidence (n_eff = 1000)
- `Beta(1, 1)`: 50% belief, zero confidence (uninformative)

In [ ]:
def visualize_prior_selection():
    """Compare different priors with same mean but different confidence"""
    
    target_mean = 0.02  # 2% belief
    
    priors = [
        ("Weak", 2, 98),         # n_eff = 100
        ("Medium", 20, 980),     # n_eff = 1000
        ("Strong", 200, 9800),   # n_eff = 10000
    ]
    
    x = np.linspace(0, 0.08, 1000)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for name, alpha, beta in priors:
        dist = stats.beta.pdf(x, alpha, beta)
        ax.plot(x, dist, linewidth=2, label=f'{name}: β({alpha}, {beta})')
        ax.fill_between(x, dist, alpha=0.2)
    
    ax.axvline(target_mean, color='black', linestyle='--', linewidth=2, 
              label='Target mean (2%)')
    
    ax.set_xlabel('Failure Rate', fontsize=12)
    ax.set_ylabel('Probability Density', fontsize=12)
    ax.set_title('Prior Selection: Same Belief, Different Confidence', 
                fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation:")
    print("="*60)
    print("Weak:   'I think it's 2%, but I'm not very sure'")
    print("        → Wide distribution, updates quickly with new data")
    print("\nMedium: 'I'm fairly confident it's around 2%'")
    print("        → Narrower distribution, moderate update rate")
    print("\nStrong: 'I'm very confident it's 2% based on years of data'")
    print("        → Very narrow distribution, updates slowly")
    print("="*60)

visualize_prior_selection()

## Takeaways for Implementation

### When to Use Bayesian Governance

✅ **Good fits**:
- High-frequency processes (daily/weekly observations)
- Critical controls where early detection matters
- Processes with historical baseline data
- Situations requiring quantified uncertainty

❌ **Poor fits**:
- Rare events (annual occurrence)
- Processes with no historical data
- Static compliance checklists
- Purely qualitative assessments

### Implementation Checklist

1. **Data infrastructure**: Can you capture daily/weekly observations?
2. **Ground truth**: Do you have labeled examples of "failure"?
3. **Prior calibration**: What's your baseline from historical audits?
4. **Threshold definition**: What PoF triggers investigation?
5. **Response protocol**: What happens when threshold is crossed?

### Common Pitfalls

- **Overconfident priors**: Using α+β >> actual sample size
- **Ignoring structural breaks**: Model assumes stationary process
- **Poor calibration**: Not validating coverage of credible intervals
- **Alert fatigue**: Setting thresholds too low
- **Lack of action**: Monitoring without governance response

---

**Remember**: Bayesian governance is not about perfect prediction. It's about **honest quantification** of what you know, what you don't know, and how your knowledge evolves as evidence accumulates.